#手写识别

In [1]:
import numpy as np
import scipy.io as sio

In [2]:
data = sio.loadmat('E:/ML_NG/04-neural network(bp)/ex4data1.mat')
raw_X = data['X']
raw_y = data['y']        #所需数据特征为X，y

In [3]:
X = np.insert(raw_X,0,values=1,axis=1)
X.shape

(5000, 401)

In [4]:
raw_y

array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]], dtype=uint8)

# 对y进行one-hot编码

In [5]:
def one_hot_encode(raw_y):
    
    result = []
    
    for i in raw_y:   # 1-10
        y_temp = np.zeros(10)
        y_temp[i-1] = 1
        
        result.append(y_temp)
        
    return np.array(result)

In [6]:
y = one_hot_encode(raw_y)
y

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [7]:
theta = sio.loadmat('E:/ML_NG/04-neural network(bp)/ex4weights.mat')  #初始权重已给出

theta1 = theta['Theta1']
theta2 = theta['Theta2']
theta1.shape,theta2.shape

((25, 401), (10, 26))

# 序列化权重参数

In [14]:
def serialize(a,b):
    
    return np.append(a.flatten(),b.flatten())

In [15]:
theta_serialize = serialize(theta1,theta2)
theta_serialize.shape

(10285,)

# 解序列化权重参数

In [22]:
def deserialize(theta_serialize):
    theta1 = theta_serialize[:25*401].reshape(25,401)
    theta2 = theta_serialize[25*401:].reshape(10,26)
    return theta1,theta2

In [23]:
theta1,theta2 = deserialize(theta_serialize)
theta1.shape,theta2.shape

((25, 401), (10, 26))

# 前向传播

In [24]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [25]:
def feed_forward(theta_serialize,X):
    theta1,theta2 = deserialize(theta_serialize)
    a1 = X
    z2 = a1 @ theta1.T
    a2 = sigmoid(z2)
    a2 = np.insert(a2,0,values=1,axis=1)
    z3 = a2 @ theta2.T
    h = sigmoid(z3)
    return a1,z2,a2,z3,h

# 损失函数

## 不带正则化

In [30]:
def cost(theta_serialize,X,y):
    a1,z2,a2,z3,h = feed_forward(theta_serialize,X)
    J = -np.sum(y*np.log(h)+(1-y)*np.log(1-h)) / len(X)
    return J

In [31]:
cost(theta_serialize,X,y)

0.2876291651613189

## 带正则化

In [35]:
def reg_cost(theta_serialize,X,y,lamda):
    sum1 = np.sum(np.power(theta1[:,1],2))
    sum2 = np.sum(np.power(theta2[:,1],2))
    reg = (sum1 + sum2) * lamda /(2*len(X))
    return reg + cost(theta_serialize, X, y)

In [36]:
lamda = 1
reg_cost(theta_serialize,X,y,lamda)

0.2890868603763652

# 反向传播

## 无正则化梯度

In [37]:
def sigmoid_gradient(z):
    return sigmoid(z)*(1-sigmoid(z))

In [38]:
def gradient(theta_serialize,X,y):
    theta1,theta2 =  cost(theta_serialize,X,y)
    a1,z2,a2,z3,h = feed_forward(theta_serialize,X)
    d3 = h - y
    d2 = da@ theta2[:,1:] * sigmoid_gradient(z2)
    D2 = (d3.T @ a2) / len(X)
    D1 = (d2.T @ a1) / len(X)
    return serialize(D1,D2)

## 带正则化梯度

In [39]:
def reg_gradient(theta_serialize,X,y,lamda):
    D = gradient(theta_serialize,X,y)
    D1,D2 = deserialize(D)
    
    theta1,theta2 =  deserialize(theta_serialize)
    D1[:,1:] = D1[:,1:] + theta[:,1:] * lamda / len(X)
    D2[:,1:] = D2[:,1:] + theta[:,1:] * lamda / len(X)
    
    return serialize(D1,D2)

# 神经网络优化

In [66]:
from scipy.optimize import minimize

def nn_training(X,y):
    
    init_theta = np.random.uniform(-0.5,0.5,10285)
    res = minimize(fun = reg_cost,
                      x0 = init_theta,
                      args = (X,y,lamda),
                      method = 'TNC',
                      jac = reg_gradient,
                  options = {'maxiter':300})

    return res

In [67]:
lamda = 10
res = nn_training(X,y)

TypeError: 'numpy.float64' object is not iterable

In [62]:
raw_y = data['y'].reshape(5000,)

In [63]:
_,_,_,_,h = feed_forward(theta_serialize,X)
y_pred = np.argmax(h,axis=1) + 1
acc = np.mean(y_pred == raw_y)

acc

0.9752

# 可视化隐藏层

In [64]:
def plot_hidden_layer(theta):
    theta1 = deserialize(theta)
    hidden_layer = theta1[:,1:] #25,400
    
    fig,ax = plt.subplots(ncols=5,nrows=5,figsize=(8,8),sharex=True,sharey=True)    #共享x，y所有属性
    
    for r in range(5):
        for c in range(5):
            ax[r,c].imshow(hidden_layer[10 * r + c].reshape(20,20).T,cmap='gray_r')
            
    plt.xticks([])
    plt.yticks([])
    
    plt.show

In [65]:
plot_hidden_layer(res.x)

NameError: name 'res' is not defined